## Capstone 1 - Part 2

In the first part of the capstone, we focused on Data Retrieval, Data Preprocessing, Feature Engineering and Exploratory Data Analysis using Python & its libraries. Now we are going to shift gears and gain insights into our HR Analytics dataset using SQL.

## TODO: Make use of SQL to do the following:

### Create a SQLITE3 DB using the CSV file (2 pts). Please refer this [link](https://www.linkedin.com/pulse/accessing-sqlite3-database-from-jupyter-notebook-using-varun-lobo/) and this [link](https://www.geeksforgeeks.org/how-to-import-csv-file-in-sqlite-database-using-python/) to know more.
### Calculate the Attrition Rate and summarize attrition (3 pts) by:
1. Gender
2. Department
3. Age
4. Average monthly income by job level
5. Years at company

### Continue using SQL to explore main reasons for attrition (3 pts), For example:
1. Why do more people over 50 years old leave the company than people who aged 40-50?
2. Why do people with higher pay still leave the company?
3. Which factors drive employees who work at company less than 5 years to leave?

### Effective Communication (2 pts)
1. Please make use of markdown cells to communicate your thought process, why did you think of performing a step? what was the observation from the query? etc.
2. The code should be commented so that it is readable for the reviewer.

### Grading and Important Instructions
1. Each of the above steps are mandatory and should be completed in good faith
2. Make sure before submitting that the code is in fully working condition
3. It is fine to make use of ChatGPT, stackoverflow type resources, just provide the reference links from where you got it
4. Debugging is an art, if you find yourself stuck with errors, take help of stackoverflow and ChatGPT to resolve the issue and if it's still unresolved, reach out to me for help.
5. You need to score atleast 7/10 to pass the project, anything less than that will be marked required, needing resubmission.
6. Feedback will be provided on 3 levels (Awesome, Suggestion, & Required). Required changes are mandatory to be made.
7. For submission, please upload the project on github and share the link to the file with us through LMS.

In [2]:
import sqlite3
import pandas as pd


df=pd.read_csv('./dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')

In [3]:
conn=sqlite3.connect('mytable.db')

In [4]:
df.to_sql(name="employee",con=conn,if_exists='replace',index=False)
conn.commit()

In [5]:
%%capture
%load_ext sql
%sql sqlite:///mytable.db

In [6]:
%%sql

select *
from employee
LIMIT 5

 * sqlite:///mytable.db
Done.


Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


In [7]:
%%sql

select CASE
        WHEN attrition = 'Yes' THEN 'True'
        WHEN attrition = 'No' THEN 'False'
        END as Attrition, 
        Round(cast(count(attrition) as float)* 100 /(select count(attrition)from employee),1) as attrition_rate
from employee
group by attrition

 * sqlite:///mytable.db
Done.


Attrition,attrition_rate
False,83.9
True,16.1


In [52]:
%%sql

select CASE
        WHEN attrition = 'Yes' THEN 'True'
        WHEN attrition = 'No' THEN 'False'
        END as Attrition,
        Gender,
        Count(attrition),
        Round(cast(count(attrition) as float)* 100/(select count(attrition) 
                                                    from employee e
                                                   where e1.gender = e.gender) ,1) as attrition_rate
from employee e1
group by attrition, gender
order by attrition desc, gender

 * sqlite:///mytable.db
Done.


Attrition,Gender,Count(attrition),attrition_rate
True,Female,87,14.8
True,Male,150,17.0
False,Female,501,85.2
False,Male,732,83.0


In [13]:
%%sql

select Department,Attrition, count(attrition) as Department_Attrition
from employee
group by attrition, department
order by department, attrition

 * sqlite:///mytable.db
Done.


Department,Attrition,Department_Attrition
Human Resources,No,51
Human Resources,Yes,12
Research & Development,No,828
Research & Development,Yes,133
Sales,No,354
Sales,Yes,92


In [61]:
%%sql

With cte as (
Select *,
    CASE
        WHEN Age <30 THEN 'Under 30'
        WHEN Age >=30 AND Age<40 Then '30 - 40'
        WHEN Age >=40 AND Age<50 THen '40 - 50'
        WHEN Age >=50 Then 'Over 50'
        End as Age_Group
from employee)

Select 
    CASE
        WHEN attrition = 'Yes' THEN 'True'
        WHEN attrition = 'No' THEN 'False'
        END as Attrition,
        Age_Group,
        Count(attrition),
        Round(cast(count(attrition) as float)* 100/(select count(attrition)
                                                           from cte c1
                                                          where c.age_group = c1.age_group) ,1) as attrition_rate
from cte c
group by age_group, attrition
order by attrition, attrition_rate desc

 * sqlite:///mytable.db
Done.


Attrition,Age_Group,Count(attrition),attrition_rate
False,40 - 50,315,90.3
False,Over 50,150,86.7
False,30 - 40,533,85.7
False,Under 30,235,72.1
True,Under 30,91,27.9
True,30 - 40,89,14.3
True,Over 50,23,13.3
True,40 - 50,34,9.7


In [37]:
%%sql

select
    Department,
    JobLevel,
    ROUND(AVG(MonthlyIncome),1) as Avg_Income,
    ROUND(AVG(Case when attrition = 'Yes' then MonthlyIncome end),1) as attrition_avg_income,
    ROUND(AVG(Case when attrition = 'Yes' then MonthlyIncome end) - AVG(MonthlyIncome),1) as difference
from employee
group by department, joblevel
having attrition_avg_income is not null

 * sqlite:///mytable.db
Done.


Department,JobLevel,Avg_Income,attrition_avg_income,difference
Human Resources,1,2733.2,2415.7,-317.5
Human Resources,3,9623.0,10216.0,593.0
Research & Development,1,2840.1,2687.4,-152.7
Research & Development,2,5291.2,5372.0,80.8
Research & Development,3,10170.5,9503.8,-666.6
Research & Development,4,15634.7,12169.0,-3465.7
Research & Development,5,19218.5,19550.0,331.5
Sales,1,2506.7,2373.4,-133.3
Sales,2,5746.1,5917.0,170.9
Sales,3,9282.3,9202.8,-79.5


In [48]:
%%sql
select 
    CASE
        WHEN yearsatcompany < 2 THEN 'New Hires'
        WHEN yearsatcompany <= 5 and yearsatcompany >= 2 THEN '2-5 years'
        WHEN yearsatcompany <= 10 and yearsatcompany >= 6 THEN '6-10 years'
        WHEN yearsatcompany <= 20 and yearsatcompany >= 11 THEN '11-20 years'
        WHEN yearsatcompany > 20 THEN 'Over 20 years'
        End as tenure_years,
        Count(attrition) as attrition_count,
        Round(cast(count(attrition) as float)* 100/(select count(attrition)from employee where attrition = 'Yes') ,1) as attrition_rate
from employee
where attrition = 'Yes'
group by tenure_years
Order By attrition_count desc

 * sqlite:///mytable.db
Done.


tenure_years,attrition_count,attrition_rate
2-5 years,87,36.7
New Hires,75,31.6
6-10 years,55,23.2
11-20 years,12,5.1
Over 20 years,8,3.4


### Continue using SQL to explore main reasons for attrition (3 pts), For example:

1. Why do more people over 50 years old leave the company than people who aged 40-50?
2. Why do people with higher pay still leave the company?
3. Which factors drive employees who work at company less than 5 years to leave?

In [106]:
%%sql

With cte as (
Select *,
    CASE
        WHEN Age >=40 AND Age<50 THen '40 - 50'
        WHEN Age >=50 Then 'Over 50'
        End as Age_Group
from employee)

Select 
    CASE
        WHEN attrition = 'Yes' THEN 'True'
        WHEN attrition = 'No' THEN 'False'
        END as Attrition,
        Age_Group,
        Round(avg(MonthlyIncome), 1) as Average_Income,
        Count(attrition) as Count_attrition,
        Round(cast(count(attrition) as float)* 100/(select count(attrition)
                                                           from cte c1
                                                          where c.age_group = c1.age_group) ,1) as attrition_rate
from cte c
where age >= 40
group by age_group, attrition
order by age_group, attrition, attrition_rate desc

 * sqlite:///mytable.db
Done.


Attrition,Age_Group,Average_Income,Count(attrition),attrition_rate
False,40 - 50,8699.0,315,90.3
True,40 - 50,7045.6,34,9.7
False,Over 50,11342.3,150,86.7
True,Over 50,8338.1,23,13.3


In [82]:
%%sql
select *
from employee
limit 5

 * sqlite:///mytable.db
Done.


Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


In [121]:
%%sql

select
    Department,
    JobSatisfaction,
    (select ROUND(AVG(MonthlyIncome),1) from employee e 
                  where e.department = e1.department and 
                  e.jobsatisfaction = e1.jobsatisfaction) as Avg_Income,
    ROUND(AVG(Case when attrition = 'Yes' then MonthlyIncome end),1) as attrition_avg_income,
    Count(attrition)
from employee e1
group by department, JobSatisfaction
having attrition_avg_income is not null and attrition_avg_income > Avg_Income

 * sqlite:///mytable.db
Done.


Department,JobSatisfaction,Avg_Income,attrition_avg_income,Count(attrition)
